In [ ]:
# Montar Drive e imports
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import cv2
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:

# Config diretórios
BASE_DIR = Path('/content/drive/MyDrive/UTP_ATM_V3')

DIR_A_PADRON = BASE_DIR / 'A_PADRONIZADO'
DIR_N_PADRON = BASE_DIR / 'N_PADRONIZADO'

DIR_A_ROI = BASE_DIR / 'A_ROI'
DIR_N_ROI = BASE_DIR / 'N_ROI'

# Criar pastas de saída se não existirem
for d in [DIR_A_ROI, DIR_N_ROI]:
    d.mkdir(parents=True, exist_ok=True)

VALID_EXTS = {'.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'}
DRY_RUN = False

LOG_FILE = BASE_DIR / 'rastreio_processamento.txt'

Mounted at /content/drive


In [ ]:
def append_log(line: str):
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        prefix = "[SIMULACAO] " if DRY_RUN else ""
        f.write(prefix + line.rstrip() + '\n')


In [ ]:
# Config ROI
# Valores do recorte
ROIS = [
    (26, 64, 340, 375),  # ROI única (x1, y1, x2, y2)
    # se quiser mais: (x1, y1, x2, y2),
]

def safe_crop(img, x1, y1, x2, y2):
    H, W = img.shape[:2]
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(W, x2)
    y2 = min(H, y2)

    if x2 <= x1 or y2 <= y1:
        return None

    return img[y1:y2, x1:x2]

In [ ]:
# Extrair ROIs
def extrair_rois(src_folder: Path, dst_folder: Path):
    """
    Lê imagens *_padronizado* em src_folder e grava ROIs em dst_folder.
    Nomes: img1A_padronizado_ROI.png (ou _ROI0, _ROI1... se houver várias).
    """
    imgs = [p for p in src_folder.glob('*_padronizado*') if p.suffix.lower() in VALID_EXTS]
    imgs.sort()
    multi = len(ROIS) > 1
    saved = skipped = 0

    for p in tqdm(imgs, desc=f"ROIs de {src_folder.name}"):
        img = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)
        if img is None:
            append_log(f"{p.stem} -> ERRO ao ler para ROI")
            continue

        for k, (x1, y1, x2, y2) in enumerate(ROIS):
            crop = safe_crop(img, x1, y1, x2, y2)

            if multi:
                out_name = f"{p.stem}_ROI{k}.png"
            else:
                out_name = f"{p.stem}_ROI.png"

            out_path = dst_folder / out_name

            if crop is not None:
                if not DRY_RUN:
                    cv2.imwrite(str(out_path), crop)
                saved += 1
                append_log(f"{p.stem} -> ROI gerada: {out_path.relative_to(BASE_DIR)}")
            else:
                skipped += 1
                append_log(f"{p.stem} -> ROI pulada (fora da imagem): {out_name}")

    print(f"\n[{dst_folder.name}] ✅ Extração de ROIs concluída")
    print(f"  • ROIs salvas:  {saved}")
    print(f"  • ROIs puladas: {skipped}")

In [ ]:
# Execução
assert DIR_A_PADRON.exists(), f"Não encontrei {DIR_A_PADRON}"
assert DIR_N_PADRON.exists(), f"Não encontrei {DIR_N_PADRON}"

extrair_rois(DIR_A_PADRON, DIR_A_ROI)
extrair_rois(DIR_N_PADRON, DIR_N_ROI)

print("\n✨ Etapa de ROIs finalizada.")
print("📄 Rastreio em:", LOG_FILE)

ROIs de A_PADRONIZADO:   0%|          | 0/100 [00:00<?, ?it/s]